#### 여기서는 Tutorial에서 배운 개념을 이용하여 간단하게 ReplayBuffer를 분산 환경에서 활용해보겠습니다. <br>즉, 아래와 같은 작업을 수행합니다. <br>  
    1. 여럿의 agent(혹은 actor)가 공유 Replay Buffer에 경험데이터를 넣는다. 
    2. Learner는 batch만큼 그 공유 ReplayBuffer에서 load한 후 원하는 작업을 수행한다.

In [1]:
import ray 
import time 
import numpy as np 

In [ ]:
ray.init() 

In [3]:
# 간단한 env를 정의하겠습니다. environment의 일반적인 메소드만 넣고 어떤 의미가 있는 행동이나 상태를 정의한 것은 아닙니다.
class Env:        
    def reset(self):
        return np.ones((2,2))
    
    def step(self, action):
        # state, reward, done 모두 random하게 지정. state의 크기는 2x2 차원을 가지는 2차원 메트릭스.
        state = action*np.random.randn(2, 2)
        reward = np.sum(state)
        
        # done은 numpy의 random.randn 이 0.06 보다 작을 때만 1을 주었습니다. 더 자주 done이 발생하도록 하고 싶다면, 0.06을 더 키우면 됩니다.
        done = 1 if abs(np.random.randn())<0.06 else 0
        return state, reward, done

In [4]:
# Buffer를 정의합니다.
class Buffer:
    def __init__(self, buffer_size):
        self.buffer_size = buffer_size
        self.state_buffer = np.zeros((buffer_size, 2 ,2))
        self.action_buffer = np.zeros(buffer_size)
        self.reward_buffer = np.zeros(buffer_size)
        self.next_state_buffer = np.zeros((buffer_size, 2 ,2))
        self.done_buffer = np.zeros(buffer_size)
        self.act_idx_buffer = np.zeros(buffer_size)
        
        self.store_idx = 0
        self.current_size = 0

    def store(self, state, action, next_state, reward, done, actor_idx):
        self.state_buffer[self.store_idx] = state
        self.action_buffer[self.store_idx] = action
        self.reward_buffer[self.store_idx] = reward
        self.next_state_buffer[self.store_idx] = next_state
        self.done_buffer[self.store_idx] = done
        self.act_idx_buffer[self.store_idx] = actor_idx
        
        self.store_idx = (self.store_idx + 1) % self.buffer_size
        self.current_size = min(self.current_size+1, self.buffer_size)
    
    def batch_load(self, batch_size): 
        indices = np.random.randint(self.store_idx, size=batch_size)  
        return dict( 
                states=self.state_buffer[indices], 
                actions=self.action_buffer[indices],
                rewards=self.reward_buffer[indices],
                next_states=self.next_state_buffer[indices], 
                dones=self.done_buffer[indices],
                actindices=self.act_idx_buffer[indices])  

In [5]:
# actor의 역할은 각각 env에서 경험한 것을 buffer에 넘겨주는 역할을 합니다.
@ray.remote
class Actor:
    def __init__(self, n_epi, learner, actor_idx):
        self.env = Env() 
        self.n_epi = n_epi # 총 episode 수 입니다.
        self.learner = learner # ray를 통해 공유하는 learner class입니다.
        self.actor_idx = actor_idx # 어떤 actor에서 온 데이터인지 보기 위한 변수입니다.

    def explore(self):
        state = self.env.reset()
        for i in range(1, self.n_epi+1):
            time.sleep(0.1)
            action = np.random.randint(10) 
            next_state, reward, done = self.env.step(action) 
            self.learner.store.remote(state, action, next_state, reward, done, self.actor_idx) 
            state = next_state
            if done:
                init_state = self.env.reset() 

In [6]:
# 공유 Buffer를 통해 학습을 진행하는 Learner를 정의합니다.
@ray.remote
class Learner:
    def __init__(self, buffer_size, batch_size):
        self.memory = Buffer(buffer_size)
        self.batch_size = batch_size
    
    # __init__에서 정의된 replay buffer에 저장합니다. 이 메소드는 각 actor마다 실행됩니다. 
    def store(self, state, action, next_state, reward, done, actor_idx):
        self.memory.store(state, action, next_state, reward, done, actor_idx)

    # 저장된 buffer에서 데이터를 로딩합니다.
    def update_network(self):
        batch = self.memory.batch_load(self.batch_size)
        loss = np.random.randn()
        buf_current_size = self.memory.store_idx
        return loss, batch['states'].shape, batch['actindices'], buf_current_size

In [7]:
buffer_size = 5000
learn_freq = 50
batch_size = 16 

learner = Learner.remote(buffer_size, batch_size) 

In [8]:
n_epi = 1000
num_actors = 16

for idx in range(num_actors): 
    globals()[f'actors_{idx}'] = Actor.remote(n_epi, learner, idx)
    globals()[f'actors_{idx}'].explore.remote()

In [ ]:
for i in range(100): 
    time.sleep(1) 
    loss, batch_stat_shape, act_indices, buf_size = ray.get(learner.update_network.remote())
    print(loss, batch_stat_shape, act_indices, buf_size)